In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_ones_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/central_pixel_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip


In [2]:
import pandas as pd
import numpy as np
import zipfile
import os
import cv2

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2, InceptionV3, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

2025-04-17 16:39:02.517132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744907942.702162      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744907942.754888      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
df = pd.read_csv("../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip", compression="zip")
df["GalaxyID"] = df["GalaxyID"].astype(str) + ".jpg"


In [7]:
filtered_df = df[(df["Class1.1"] >= 0.8) | (df["Class1.2"] >= 0.8)].copy()
filtered_df.drop_duplicates(inplace=True)
filtered_df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
4,100090.jpg,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
8,100134.jpg,0.021834,0.976952,0.001214,0.021751,0.955201,0.313077,0.642124,0.546491,0.408711,...,0.021751,0.207253,0.152044,0.187194,0.000000,0.00000,0.054649,0.081974,0.081974,0.327894
14,100237.jpg,0.164391,0.800759,0.034850,0.030795,0.769964,0.149037,0.620928,0.387742,0.382223,...,0.000000,0.208685,0.179056,0.000000,0.032728,0.14098,0.000000,0.000000,0.089185,0.124848
16,100263.jpg,0.179654,0.818530,0.001816,0.573791,0.244739,0.047326,0.197413,0.016623,0.228116,...,0.141605,0.000000,0.007855,0.008768,0.000000,0.00000,0.000000,0.000000,0.000000,0.016623
19,100322.jpg,0.091987,0.908013,0.000000,0.816806,0.091207,0.000000,0.091207,0.000000,0.091207,...,0.142815,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [8]:
filtered_df["label"] = np.where(filtered_df["Class1.1"] >= 0.8, "elliptical", 
                                np.where(filtered_df["Class1.2"] >= 0.8, "spiral", "uncertain"))


In [9]:
spiral_galaxies = filtered_df[filtered_df["label"] == "spiral"].sample(8000, random_state=1)
elliptical_galaxies = filtered_df[filtered_df["label"] == "elliptical"].sample(8000, random_state=1)
combined_df = pd.concat([spiral_galaxies, elliptical_galaxies])[["GalaxyID", "label"]]


In [10]:
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [11]:
image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [12]:
train_flow = image_gen.flow_from_dataframe(
    dataframe=combined_df,
    directory="./images_training_rev1",
    x_col="GalaxyID",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=123
)

Found 12800 validated image filenames belonging to 2 classes.


In [13]:
val_flow = image_gen.flow_from_dataframe(
    dataframe=combined_df,
    directory="./images_training_rev1",
    x_col="GalaxyID",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=True,
    seed=123
)

Found 3200 validated image filenames belonging to 2 classes.


In [14]:
mobilenet_base = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
for layer in mobilenet_base.layers[-10:]:
    layer.trainable = True


I0000 00:00:1744908059.752275      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [15]:
mobilenet_model = Sequential([
    mobilenet_base,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])


In [16]:
mobilenet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
mobilenet_model.fit(train_flow, validation_data=val_flow, epochs=10)
A

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744908122.790614      76 service.cc:148] XLA service 0x7f72340020c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744908122.791429      76 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1744908126.047038      76 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1744908131.528927      76 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1744908131.726222      76 gpu_timer.cc:

  1/400 ━━━━━━━━━━━━━━━━━━━━ 6:08:23 55s/step - accuracy: 0.5938 - loss: 0.6788

I0000 00:00:1744908142.226070      76 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


400/400 ━━━━━━━━━━━━━━━━━━━━ 96s 102ms/step - accuracy: 0.8725 - loss: 0.2972 - val_accuracy: 0.0584 - val_loss: 4.7662
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.9668 - loss: 0.0887 - val_accuracy: 0.1659 - val_loss: 4.6193
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.9832 - loss: 0.0437 - val_accuracy: 0.3953 - val_loss: 2.0503
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.9873 - loss: 0.0358 - val_accuracy: 0.5016 - val_loss: 1.3845
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 89ms/step - accuracy: 0.9921 - loss: 0.0244 - val_accuracy: 0.2281 - val_loss: 4.6742
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 89ms/step - accuracy: 0.9944 - loss: 0.0138 - val_accuracy: 0.4122 - val_loss: 4.8251
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.9937 - loss: 0.0205 - val_accuracy: 0.6806 - val_loss: 1.4925
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 37s 91ms/step - accuracy: 0.9945 - loss: 0.0148 - val_accura

In [17]:
inception_base = InceptionV3(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
for layer in inception_base.layers[-10:]:
    layer.trainable = True


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
inception_model = Sequential([
    inception_base,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])


In [19]:
inception_model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
inception_model.fit(train_flow, validation_data=val_flow, epochs=10)


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 152s 168ms/step - accuracy: 0.8171 - loss: 0.4035 - val_accuracy: 0.9553 - val_loss: 0.1306
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 61s 151ms/step - accuracy: 0.9640 - loss: 0.1061 - val_accuracy: 0.9659 - val_loss: 0.0970
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 61s 151ms/step - accuracy: 0.9814 - loss: 0.0531 - val_accuracy: 0.9572 - val_loss: 0.1168
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 60s 150ms/step - accuracy: 0.9888 - loss: 0.0337 - val_accuracy: 0.9737 - val_loss: 0.0719
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 62s 154ms/step - accuracy: 0.9945 - loss: 0.0167 - val_accuracy: 0.9644 - val_loss: 0.1074
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 61s 151ms/step - accuracy: 0.9952 - loss: 0.0156 - val_accuracy: 0.9712 - val_loss: 0.0845
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 61s 151ms/step - accuracy: 0.9983 - loss: 0.0069 - val_accuracy: 0.9669 - val_loss: 0.1106
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 60s 150ms/step - accuracy: 0.9977 - loss: 